In [ ]:
# !pip install -U textblob
!python -m textblob.download_corpora lite

In [ ]:
from textblob import TextBlob

In [3]:
import json
from tqdm import tqdm

In [4]:
file_d = []
reviews_sent = []
reviews_neg_sent = []
reviews_str = ""
reviews_pos_str = ""
reviews_neg_str = ""

In [1]:
file = '/Users/gkhanna/Downloads/reviews_Home_and_Kitchen_5.json'

Reading "count" of the file for faster experiments
count = 0 pulls in all data

In [5]:
# Reading count number of JSON lines from the file
count = 0
n = 0
with open(file, "r") as f:
    for line in tqdm(f):
        file_d.append(json.loads(line))
        n =  n + 1
        if count > 0 and n == count:
            break
    

551682it [00:06, 82919.57it/s] 


In [6]:
# Checking the structures
print(len(file_d))
print(type(file_d))
print(file_d[0])
print(type(file_d[0]))

551682
<class 'list'>
{'reviewerID': 'APYOBQE6M18AA', 'asin': '0615391206', 'reviewerName': 'Martin Schwartz', 'helpful': [0, 0], 'reviewText': 'My daughter wanted this book and the price on Amazon was the best.  She has already tried one recipe a day after receiving the book.  She seems happy with it.', 'overall': 5.0, 'summary': 'Best Price', 'unixReviewTime': 1382140800, 'reviewTime': '10 19, 2013'}
<class 'dict'>


Read in reviews that correspond to the list of ASIN's (Amazon product ID's) selected. 
Choosing the ASIN that has ~ 1000 reviews from the EDA

In [7]:
# ASIN corresponding to the Iron Skillet
pl = ['B00006JSUA']

In [8]:
# Getting the reviews out of the dictionary
# into a list and string
for r in tqdm(file_d):
    if r['asin'] in pl:
        reviews_sent.append(r['reviewText'])
        reviews_str = reviews_str + str(r['reviewText'])
        if ((r['overall'] == 1.0) or (r['overall'] == 2.0)):
            reviews_neg_str = reviews_neg_str + str(r['reviewText'])
        else:
            reviews_pos_str = reviews_pos_str + str(r['reviewText'])

100%|██████████| 551682/551682 [00:00<00:00, 1738950.28it/s]


In [9]:
len(reviews_str)

598363

In [ ]:
blob = TextBlob(reviews_str)

In [ ]:
reviews_sent[0]

In [ ]:
blob[0]

In [ ]:
type(blob)

In [ ]:
len(blob)

In [ ]:
blob_s = blob.sentences

In [ ]:
type(blob_s)

In [ ]:
len(blob_s)

In [ ]:
type(blob_s[0])

In [ ]:
blob_s[0]

In [ ]:
blob_s[2]

In [ ]:
import nltk

Trying out different noun phrase extractors

In [ ]:
from textblob.np_extractors import ConllExtractor
extractor = ConllExtractor()

In [ ]:
blob_nc = blob.noun_phrases

In [ ]:
blob_nc[:10]

In [ ]:
freqdist = nltk.FreqDist(word for word in blob_nc)
most_common = freqdist.most_common()
print(most_common[:20])

A lot of useless stuff in the text

How's the library in finding the sentiment of sentences

In [ ]:
blob_s[0].sentiment

In [ ]:
blob_s[2].sentiment

Trying sentence summarization using gensim

In [ ]:
# !pip install --upgrade gensim

In [ ]:
from gensim.summarization import summarize

In [ ]:
print(file_d[0]['reviewText'])

In [ ]:
print(summarize(file_d[0]['reviewText']))

In [ ]:
print(file_d[1]['reviewText'])

In [ ]:
print(summarize(reviews_str))

In [ ]:
len(summarize(reviews_str))

In [ ]:
# !pip install git+https://github.com/boudinfl/pke.git

In [ ]:

# !python -m nltk.downloader stopwords
# !python -m nltk.downloader universal_tagset
# !python -m spacy download en # download the english model


Keyphrase extraction using PKE

In [10]:
import pke

In [11]:
# initialize keyphrase extraction model, here TopicRank
extractor = pke.unsupervised.TopicRank()

In [12]:
# Load text
#  preprocessing is carried out using spacy
extractor.load_document(input=reviews_str, language='en', normalization = 'stemming')

In [ ]:
# keyphrase candidate selection, in the case of TopicRank: sequences of nouns
# and adjectives (i.e. `(Noun|Adj)*`)
extractor.candidate_selection()

In [ ]:
# candidate weighting, in the case of TopicRank: using a random walk algorithm
extractor.candidate_weighting()

In [ ]:
# print the n-highest (10) scored candidates
for (keyphrase, score) in extractor.get_n_best(n=10, stemming=True):
    print(keyphrase, score)

Longest Sequence of Nouns and Adjectives

In [13]:
# select the keyphrase candidates, for TopicRank the longest sequences of 
# nouns and adjectives
extractor.candidate_selection(pos={'NOUN', 'PROPN', 'ADJ'})

In [14]:
# weight the candidates using a random walk. The threshold parameter sets the
# minimum similarity for clustering, and the method parameter defines the 
# linkage method
extractor.candidate_weighting(threshold=0.50, method='average')

In [15]:
# print the n-highest (10) scored candidates
for (keyphrase, score) in extractor.get_n_best(n=10, stemming=True):
    print(keyphrase, score)

pan 0.03418016185348628
skillet 0.021094132395246244
cast iron 0.015084633950159377
egg 0.009945949128224675
food 0.008314118391616287
lodg 0.007986959651326604
time 0.007975499927931302
great 0.007165577854183732
oil 0.006945493434940615
oven 0.006789662547419141
